In [1]:
#!/usr/bin/env python
# coding: utf-8

# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

from reader import *
from string_matching_by_word import *
from string_matching_by_spaCy_NP import *
from word_embeddings_by_spaCy_NP import *
from hobbs import entry
from nltk import pos_tag

import sys
import contextlib

import warnings
warnings.filterwarnings("ignore")

# ### Read Document

# input_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/{}.input'.format(file_number)
# key_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/{}.key'.format(file_number)

In [7]:
# if __name__ == "__main__":
    
#     input_path_file = sys.argv[1]
#     output_path = sys.argv[2]
input_path_file = 'input.txt' 
output_path = 'output_responses/'
# driver(input_path_file, output_path)
    

In [16]:
coref_dict_match_NP

{'X0': [['PANTEX', 2, 'TEX', 100],
  ['PANTEX', 5, 'BC-PANTEX-FLIGHTS-TEX', 100],
  ['PANTEX', 11, 'the Pantex Weapons Plant', 100],
  ['PANTEX', 14, 'Pantex', 100],
  ['PANTEX', 21, 'Pantex', 100]],
 'X1': [['FLIGHTS', 5, 'BC-PANTEX-FLIGHTS-TEX', 100],
  ['FLIGHTS', 13, 'flight', 100],
  ['FLIGHTS', 16, 'current flight data', 86],
  ['FLIGHTS', 25, 'a flight pattern', 86],
  ['FLIGHTS', 27, 'flight paths', 86],
  ['FLIGHTS', 29, 'The second flight path', 86]],
 'X3': [['FAA', 12, 'The FAA', 100],
  ['FAA', 19, 'the FAA air traffic manager', 100]],
 'X4': [['NUMBER OF FLIGHTS', 13, 'flight', 100]],
 'X5': [['PLUTONIUM', 11, "the nation's surplus plutonium", 100],
  ['PLUTONIUM', 14, 'a plutonium storage bunker', 100],
  ['PLUTONIUM', 15, 'Plutonium', 100],
  ['PLUTONIUM', 20, 'plutonium', 100],
  ['PLUTONIUM', 22, 'The hollow plutonium ball', 100],
  ['PLUTONIUM', 31, 'the plutonium area', 100]],
 'X7': [['Amarillo', 17, 'Amarillo', 100],
  ['Amarillo', 19, 'Amarillo International', 10

In [17]:
coref_dict_all_sorted

{'X1': [['FLIGHTS', 13, 'flight', 0.8170893645060706],
  ['FLIGHTS', 27, "the airport's dozen flight paths", 0.641786076416007],
  ['FLIGHTS', 17, 'the airport', 0.6281263860060029],
  ['FLIGHTS', 25, 'the airport traffic', 0.5671975117171633],
  ['FLIGHTS', 16, 'current flight data', 0.5661745059456174],
  ['FLIGHTS', 11, 'aircraft', 0.557844332316427],
  ['FLIGHTS', 23, 'The Amarillo airport', 0.5504458349817198],
  ['FLIGHTS', 26, 'planes', 0.5388332589536257],
  ['FLIGHTS', 25, 'a flight pattern', 0.5163587228224844],
  ['FLIGHTS', 29, 'The second flight path', 0.5121068765000295]],
 'X2': [['02-14', 5, 'BC-PANTEX-FLIGHTS-TEX', 0.5648850173281347]],
 'X3': [['FAA', 12, 'The FAA', 0.8408831230057551]],
 'X4': [['NUMBER OF FLIGHTS',
   27,
   "the airport's dozen flight paths",
   0.7848771092478396],
  ['NUMBER OF FLIGHTS', 11, 'the number', 0.7408435662712148],
  ['NUMBER OF FLIGHTS', 16, 'current flight data', 0.7288887772330651],
  ['NUMBER OF FLIGHTS', 17, 'the airport', 0.71660

In [8]:
# def driver(input_path_file, output_path):
    
fileObj = open(input_path_file,'r')
list_of_ip_path = fileObj.readlines()
list_of_ip_path = list(map(lambda x: x.rstrip('\n'), list_of_ip_path))

for input_path in list_of_ip_path:

    file_number = input_path.split('/')[-1].split('.')[0]

    print('Processing file {}.input...'.format(file_number))

    read = ReadInput(input_path)
    list_of_sentences = read.getListOfSentences()
    full_text = read.getText()

    # ### Get Sentence Dict and Cluster Head Dict

    sentence_dict = getSentenceDict(list_of_sentences)
    cluster_head_dict = getClusterHeads(sentence_dict)

    # ### Get Coref for exact match by word

#         coref_dict_match_word = getCorefDict_match_word(sentence_dict, cluster_head_dict,90)

    # ### Get Coref for threshold match by NP

    coref_dict_match_NP = getCorefDict_match_NP(sentence_dict, cluster_head_dict,80)

    # ### Get Coref for Word Embedding Similarity by NP

    coref_dict_all_sorted = getCorefDict_all_sorted(sentence_dict, cluster_head_dict,0.5)
    # coref_dict_max_sentence = getCorefDict_max_of_each_sentence(sentence_dict, cluster_head_dict,0.5)

    # coref_dict_all_sorted_top3 = get_TopN_Matches(coref_dict_all_sorted,3)

    # ### Merging coref_dict_match_NP with coref_dict_all_sorted_top3


    def mergeDicts(string_matching_dict, word_embedding_dict):

        d1 = string_matching_dict.copy()
        d2 = word_embedding_dict.copy()

        d1_keys = d1.keys()
        d2_keys = d2.keys()

        for key in d1_keys:
            d1[key] = list(map(lambda x: x[0:3], d1[key]))

        for key in d2_keys:
            d2[key] = list(map(lambda x: x[0:3], d2[key]))

        for key in d2_keys:
            if(key not in d1_keys):
                d1[key] = d2[key]
            else:
                for val in d2[key]:
                    if(val not in d1[key]):
                        d1[key].append(val)

        return d1


    coref_final = mergeDicts(coref_dict_match_NP, coref_dict_all_sorted)


    # ### Passing the reference dict to Hobbs

    coref_final_with_pro = entry(list_of_sentences, cluster_head_dict, coref_final)

    # ### Adding those Cluster Heads which were not predicted

    def addNOPredsClusterHeads(cluster_head_dict, coref_final_with_pro):
        for cluster_id, cluster_val in cluster_head_dict.items():
            if(cluster_id not in coref_final_with_pro.keys()):
                coref_final_with_pro[cluster_id] = [['',int(cluster_val[0]),'', None]]

        return coref_final_with_pro

    coref_final_with_pro = addNOPredsClusterHeads(cluster_head_dict, coref_final_with_pro)

    # ### Removing the Determiners from the start

#         def removeDetfromStart(coref_final_with_pro):
#             for cluster in coref_final_with_pro.keys():
#                 for i in range(0, len(coref_final_with_pro[cluster])):
#                     if(len(coref_final_with_pro[cluster][i][2].split(' '))>1):
#                         ref_word_list = coref_final_with_pro[cluster][i][2].strip().split(' ')
#                         ref_word_list = list(filter(bool, ref_word_list) )
#                         check = pos_tag(ref_word_list)
#                         if len(check) >1:
#                             if 'DT' in check[0][1]:
#                                 sentence = ' '.join(coref_final_with_pro[cluster][i][2].split(' ')[1:])
#                                 coref_final_with_pro[cluster][i][2] = ' '.join(coref_final_with_pro[cluster][i][2].split(' ')[1:])

#                     coref_final_with_pro[cluster][i][2] = coref_final_with_pro[cluster][i][2].split(' ')[-1]

#             return coref_final_with_pro

#         coref_final_with_pro = removeDetfromStart(coref_final_with_pro)

    # ### Making no predictions for Pronouns (will be included in final version - Reverse Hobb's/ Trained Word Embedding model)

    def removePredsforPronouns(coref_final_with_pro):
        for cluster in coref_final_with_pro.keys():
            for i in range(0, len(coref_final_with_pro[cluster])):
                curr_cluster_name = coref_final_with_pro[cluster][i][0].split(' ')
                if(len(curr_cluster_name)==1 and curr_cluster_name[0]!=''):
                    if('PRP' in pos_tag(curr_cluster_name)[0]):
                        coref_final_with_pro[cluster] = []
                        break
        return coref_final_with_pro

    # In[396]:

    coref_final_with_pro = removePredsforPronouns(coref_final_with_pro)

    # ### Print Output

    def printOP(cluster_head_dict, coref_final_with_pro):

        for cluster_id, cluster_head_name in cluster_head_dict.items():

            print('<COREF ID="{}">{}</COREF>'.format(cluster_id, cluster_head_name[1]))

            coreferences = coref_final_with_pro[cluster_id]
            list_of_sent_ids = list(map(lambda x: x[1], coreferences))
            sorted_index_sent_ids = [i[0] for i in sorted(enumerate(list_of_sent_ids), key=lambda x:x[1])]
            coreferences = [coreferences[i] for i in sorted_index_sent_ids]

            for coref in coreferences:
                if(coref[0] == ''):
                    continue
                print('{{{0}}}'.format(coref[1]) + ' ' + '{' + coref[2] + '}')
            print('\n', end = '')      

#         with open(output_path+'/{}.response'.format(file_number),'w') as f:
#             with contextlib.redirect_stdout(f):
#                 printOP(cluster_head_dict, coref_final_with_pro)

    print('Output saved for file {}.input!'.format(file_number))

    printOP(cluster_head_dict, coref_final_with_pro)
    break

Processing file a8.input...
Output saved for file a8.input!
<COREF ID="X0">PANTEX</COREF>
{2} {TEX}
{5} {BC-PANTEX-FLIGHTS-TEX}
{11} {the Pantex Weapons Plant}
{14} {Pantex}
{21} {Pantex}

<COREF ID="X1">FLIGHTS</COREF>
{5} {BC-PANTEX-FLIGHTS-TEX}
{11} {aircraft}
{13} {flight}
{16} {current flight data}
{17} {the airport}
{18} {We}
{23} {The Amarillo airport}
{25} {a flight pattern}
{25} {the airport traffic}
{26} {planes}
{27} {flight paths}
{27} {the airport's dozen flight paths}
{29} {The second flight path}

<COREF ID="X2">02-14</COREF>
{5} {BC-PANTEX-FLIGHTS-TEX}

<COREF ID="X3">FAA</COREF>
{12} {The FAA}
{19} {the FAA air traffic manager}

<COREF ID="X4">NUMBER OF FLIGHTS</COREF>
{11} {the number}
{11} {The Federal Aviation Administration}
{11} {aircraft}
{13} {flight}
{13} {up to 30 passes}
{14} {a plane crash}
{16} {current flight data}
{16} {one chance}
{17} {the airport}
{18} {these planes}
{19} {the FAA air traffic manager}
{20} {25 planes}
{21} {the Energy Department}
{21} 